# oemof tabular

In [1]:
import os
import pkg_resources as pkg
import pandas as pd
import numpy as np 
import math

from pyomo.opt import SolverFactory
import pyomo.environ as po 
from oemof.solph import EnergySystem, Model, Bus
from oemof.tools.economics import annuity as annuity
from oemof.solph import constraints
import oemof.tabular.tools.postprocessing as pp
import oemof.tabular.facades as fc

In [2]:
scenario = "RR" # name of the scenario and the corresponding file, e.g. "A.xls"
carrier_scenario = "base"
tech_scenario = "base"

country = "barbados"

## Creating and Setting the Datapaths

First, the datapath for raw-data and results is set. Data handling looks more complex than it is. You can easily adapt this to a simple `pd.read_excel(filepath,...)` in the next block if your file is located somewhere else. Otherwise we will use data from the repository repository. 

In addition a results directory will be created in `home/user/oemof-results/results/<scenario-name>/output`. 

In [3]:
# datapath for input data from the oemof tabular pacakge
datapath = os.path.join(
    os.getcwd(),#pkg.resource_filename("oemof.jordan", ""),
    "scenarios", 
    scenario + ".xls",
)

carrier_technology_path = os.path.join(
    os.getcwd(), 
    "scenarios",
    "carrier-technology.xls"
)

# results path
results_path = os.path.join(
    os.path.expanduser("~"), "oemof-results", country,
)

scenario_path = os.path.join(
    results_path, ("-").join([scenario])#, carrier_scenario, tech_scenario])
)

if not os.path.exists(scenario_path):
    os.makedirs(scenario_path)

Next the required input data will be read. The profiles index will be used for the `EnergySystem` object below. 
All generator data etc. will also be loaded. 

In [4]:
file = pd.ExcelFile(datapath)


technology = pd.read_excel(
    carrier_technology_path, sheet_name="technology-data", index_col=[0, 1, 2]
)
carrier = pd.read_excel(
    carrier_technology_path, sheet_name="carrier", index_col=[0, 1]
)

sheet_names = [typ for typ in file.sheet_names if typ in fc.TYPEMAP.keys()]


data = {}

for sheet in sheet_names: 
    data[sheet]= pd.read_excel(datapath, sheet_name=sheet, index_col=0)
    # add effiency if not set 
    if "efficiency" in data[sheet].columns:
        eta = []
        for _, row in data[sheet].iterrows():
            if math.isnan(row["efficiency"]):
                eta.append(
                    technology.to_dict()["efficiency"].get((row["carrier"], row["tech"], tech_scenario), 
                                                          float("nan")))
            else:
                eta.append(row["efficiency"])
        data[sheet]["efficiency"] = eta

# profiles and tech data not be part of TYPEMAP and need to be read seperately
profiles = pd.read_excel(
    datapath,
    sheet_name="profiles",
    index_col=[0],
    parse_dates=True,
)
profiles.index.freq = "1H"



if "co2-limit" in file.sheet_names:
    co2_limit = pd.read_excel(datapath, sheet_name="co2-limit", index_col=0)
else: 
    co2_limit = None


In [5]:
all_components = pd.concat([v for k, v in data.items() if k not in ["bus", "co2-limit"]], sort=False)
# Only be used for Latex export of tables 
#columns = ['profile', 'capacity_potential']
#print(all_components.to_latex(columns=columns, na_rep="-"))

In [6]:
def _capacity_cost(g, tech_scenario=tech_scenario):
    if bool(g["expandable"]):
        val = annuity(
            technology.at[(g["carrier"], g["tech"], tech_scenario), "capex"], 
            technology.at[(g["carrier"], g["tech"], tech_scenario), "lifetime"],
            technology.at[(g["carrier"], g["tech"], tech_scenario), "wacc"]) * (
            (1 + (technology.at[(g["carrier"], g["tech"], tech_scenario), "fom"])
            )) 
        return val * 1000 # $/kw -> $/MW
    else:
        return None
    
def _marginal_cost(g, tech_scenario, carrier_scenario):
    if not isinstance(g, dict):
        g = g.to_dict()
    if not pd.isna(g.get("marginal_cost", np.nan)):
        return g["marginal_cost"]
    else:
        return (
            (carrier.at[(g["carrier"], carrier_scenario), "cost"] / 
             technology.at[(g["carrier"], g["tech"], tech_scenario), "efficiency"]) + 
            technology.at[(g["carrier"], g["tech"], tech_scenario), "vom"]
            )
    
def _none(number):
    if pd.isna(number):
        return None
    else:
        return number
    
def _write_investment_cost(scenario_path):
    invest_e = {}
    invest_p = {}
    for n in es.nodes: 
        if hasattr(n, "storage_capacity_cost"):
            if n.storage_capacity_cost is not None:
                invest_e[n.label] = (n.storage_capacity_cost, m.results[n, None]["scalars"]["invest"])
                invest_p[n.label] = (n.capacity_cost, m.results[es.groups["BB-electricity"], n]["scalars"]["invest"])
        elif hasattr(n, "capacity_cost"):
            if n.capacity_cost is not None:
                invest_p[n.label] = (n.capacity_cost, m.results[n, es.groups["BB-electricity"]]["scalars"]["invest"])

        pd.DataFrame(invest_e, index=["$/MWha", "MWh"]).to_csv(
            os.path.join(scenario_path, "investment_energy.csv"))
        pd.DataFrame(invest_p, index=["$/MWa", "MW"]).to_csv(
            os.path.join(scenario_path, "investment_power.csv"))

## Creating the EnergySystem and its Nodes

Firs, an `EnergySystem` object will be created. This holds all information (nodes, etc.) of the modelled energy system that will be added below. This is just the standard way of using the `oemof.solph` library for your modelling.

In [7]:
es = EnergySystem(timeindex=profiles.index)

### Buses

Before any components are added, all bus objects are added tothe energy system object.

In [8]:
buses = {
    name: Bus(label=name, balanced=bool(arg.balanced))
    for name, arg in data["bus"].iterrows()
}
es.add(*buses.values())


#### Bus Constraints 

With the set of all Buses $B$ all inputs $x^{flow}_{i(b),b}$ to a bus $b$ must equal all its outputs $x^{flow}_{b,o(b)}$

$$\sum_i x^{flow}_{i(b), b}(t) - \sum_o x^{flow}_{b, o(b)}(t) = 0 \qquad \forall t \in T, \forall b \in B$$

This equation will be build once the complete energy system is setup with its components. Every time a `Component` is created, the connected bus inputs/outputs will be updated. By this update every bus has all required information of its inputs and outputs available to construct the constraints. 


### Load components

#### Load Constraint 

For the set of all Load denoted with $l \in L$ the load $x_l$ at timestep t equals the exogenously defined  profile value $c^{profile}_l$ multiplied by the amount of this load $c^{amount}_l$

$$ x^{flow}_{l}(t) = c^{profile}_{l}(t) \cdot c^{amount}_{l} \qquad \forall t \in T, \forall l \in L$$

In [9]:
for name, l in data["load"].iterrows():
    es.add(
        fc.Load(
            label=name,
            bus=buses[
                l.bus
            ],  # reference the bus in the buses dictionary
            amount=l.amount,  # amount column
            profile=profiles[l.profile],
        )
    )


/home/admin/projects/oemof-barbados/barbados-env/lib/python3.7/site-packages/oemof/solph/network.py:250: SuspiciousUsageWarning: `Sink` 'el-load' constructed without `inputs`.
  warn(msg.format(self), debugging.SuspiciousUsageWarning)
/home/admin/projects/oemof-barbados/barbados-env/lib/python3.7/site-packages/oemof/solph/network.py:250: SuspiciousUsageWarning: `Sink` 'ev-load' constructed without `inputs`.
  warn(msg.format(self), debugging.SuspiciousUsageWarning)
/home/admin/projects/oemof-barbados/barbados-env/lib/python3.7/site-packages/oemof/solph/network.py:250: SuspiciousUsageWarning: `Sink` 'cruise-load' constructed without `inputs`.
  warn(msg.format(self), debugging.SuspiciousUsageWarning)


### Dispatchable and Volatile component

#### Dispatchable Generator Constraint

A `Dispatachble` component can be used to model all types of dispatchble units in a energy system. This can include diesel generators oder coal fired power plants but also hot water boilers for heat. Every generator **must** be connected to an `Bus` object. 

This basic mathematical model for the component with the set of all dispatchable generators being $d \in D$ looks as follows:

$$x^{flow}_{d}(t) \leq c^{capacity}_{d} \qquad \forall t \in T,  \forall d \in D$$

Meaning, the production of the generator $x^{flow}$ must be less than its maximum capacity $c^{capacity}_d$ in every timestep. *Note that this equation holds for the case where the `expandable` attribute is set to `False`*. For the investment case the following two equation hold:

$$x^{flow}_{d}(t) \leq x^{capacity}_{d} \qquad \forall t \in T,  \forall d \in D$$
$$c^{capacity}_d \leq x^{capacity}_{d} \leq c^{capacity\_potential}_{d} \forall d \in D$$


In [10]:

for name, g in data["dispatchable"].iterrows():
    g = g.to_dict()
    es.add(
        fc.Dispatchable(
            label=name,
            bus=buses[g["bus"]],
            carrier=g["carrier"],
            tech=g["tech"],
            marginal_cost=_marginal_cost(g, tech_scenario, carrier_scenario),
            expandable=g.get("expandable", False),
            capacity=g["capacity"],
            capacity_potential=None,
            capacity_cost=_capacity_cost(g, tech_scenario),
            output_parameters={
                "emission_factor": (
                    carrier.at[(g["carrier"], carrier_scenario), "emission_factor"]
                    / g["efficiency"]
                ),
               "summed_max": technology.at[
                    (g["carrier"], g["tech"], tech_scenario), "avf"
                ] * 8760,
            },
        )
    )

/home/admin/projects/oemof-barbados/barbados-env/lib/python3.7/site-packages/oemof/solph/network.py:263: SuspiciousUsageWarning: `Source` 'hfo-lsce' constructed without `outputs`.
  warn(msg.format(self), debugging.SuspiciousUsageWarning)
/home/admin/projects/oemof-barbados/barbados-env/lib/python3.7/site-packages/oemof/solph/network.py:263: SuspiciousUsageWarning: `Source` 'hfo-msce' constructed without `outputs`.
  warn(msg.format(self), debugging.SuspiciousUsageWarning)


#### Volatile Generator Constraint

For the `Volatile` $v \in V$ generators that can be used to model PV and Wind units the flow will be fixed by a given profile multiplied with the (optimised) capacity $x^{capacity}_{v}$ in the investment case and the exougenously defined capacity $c^{capacity}_{v}$ in the non-investment case (i.e. `expandable` attribute `False`). 

$$ x^{flow}_{v}(t) = c^{profile}_{v}(t) \cdot x^{capacity}_{v} \qquad \forall t \in T, \forall v \in V$$

The optimised capacity is limited by the following variable bounds:

$$c^{capacity}_v \leq x^{capacity}_{v} \leq c^{capacity\_potential}_{v} \forall v \in V$$


In [11]:
for name, v in data["volatile"].iterrows():
    es.add(
        fc.Volatile(
            label=name,
            bus=buses[v.bus],
            carrier=v.carrier,
            tech=v.tech,
            expandable=v.expandable,
            capacity=v.capacity,
            capacity_potential=v.capacity_potential,
            capacity_cost=_capacity_cost(v, tech_scenario),
            profile=profiles[v.profile],
        )
    )


/home/admin/projects/oemof-barbados/barbados-env/lib/python3.7/site-packages/oemof/solph/network.py:263: SuspiciousUsageWarning: `Source` 'solar-pv-distributed' constructed without `outputs`.
  warn(msg.format(self), debugging.SuspiciousUsageWarning)
/home/admin/projects/oemof-barbados/barbados-env/lib/python3.7/site-packages/oemof/solph/network.py:263: SuspiciousUsageWarning: `Source` 'solar-pv-utility' constructed without `outputs`.
  warn(msg.format(self), debugging.SuspiciousUsageWarning)
/home/admin/projects/oemof-barbados/barbados-env/lib/python3.7/site-packages/oemof/solph/network.py:263: SuspiciousUsageWarning: `Source` 'wind-onshore' constructed without `outputs`.
  warn(msg.format(self), debugging.SuspiciousUsageWarning)


### Storage components

In [12]:
for name, s in data["storage"].iterrows():
    s = s.to_dict()
    es.add(
        fc.Storage(
            label=name,
            bus=buses[s["bus"]],
            carrier=s["carrier"],
            tech=s["tech"],
            marginal_cost=s["marginal_cost"],
            capacity=s["capacity"],
            storage_capacity=s["storage_capacity"],
            storage_capacity_potential=s["storage_capacity_potential"],
            min_storage_level=s.get("min_storage_level", 0),
            expandable=s["expandable"],
            efficiency=s["efficiency"],
            loss_rate=s["loss_rate"],
            initial_storage_level=s["initial_storage_level"],
            invest_relation_output_capacity=_none(s.get("invest_relation_output_capacity")),
            invest_relation_input_capacity=_none(s.get("invest_relation_input_capacity")),
            storage_capacity_cost=annuity(
                technology.at[
                    (s["carrier"], s["tech"], tech_scenario), "storage_capex"
                ],
                technology.at[
                    (s["carrier"], s["tech"], tech_scenario), "lifetime"
                ],
                 technology.at[
                    (s["carrier"], s["tech"], tech_scenario), "wacc"
                ],
            )
            * 1000,  # $/kW -> $/MW
            capacity_cost=_capacity_cost(s, tech_scenario)
    )
)

#### Storage Constraints 

The mathematical representation of the storage for all storages $s \in S$ will include the flow into the storage, out of the storage and a storage level. The default efficiency for input/output is 1. Note that the provided efficiency is applied for charge and discharge. Hence, if you want to set the round trip efficiency you need to set $\eta = \sqrt{\eta^{roundtrip}}$, where $\eta$ is the efficiency provided to construct the component. 

Intertemporal energy balance of the storage:

$$ x^{level}_{s}(t) = (1-\eta^{loss\_rate}) x^{level}_{s}(t) + \eta_{in} x^{flow}_{s, in} - \frac{x^{flow}_{s, out}(t)}{\eta_{out}} \qquad \forall t \in T,  \forall s \in S$$ 

Bounds of the storage level variable $x^{level}_s(t)$ with investment. The first and the last timestep are linked and set to a fixed value (e.g. 0.5 of the maximum storage capacity)

$$ x^{level}_s(1) = x_s^{level}(t_{e}) = 0.5 \cdot \overline{x}_s^{level} \qquad \forall t \in T,  \forall s \in S$$ 

Additionally, the level is bounded by the optimised storage capacity (energy) $\overline{x}_s^{level}$:

$$ x^{level}_s(t) \leq \overline{x}_s^{level} \qquad \forall t \in T, \forall s \in S$$ 

The investment itself is limited by parameters of the existing storage capacity and its potential: 

$$ c_s^{storage\_capacity} \leq \overline{x}_s^{level} \leq c_s^{storage\_capacity\_potential} $$

The inflow/outflow (i.e. power) of the storage also needs to be within the limit of the minimum and maximum power. 

$$ -x_s^{capacity} \leq x^{flow}_s(t) \leq x_s^{capacity} \qquad \forall t \in T, \forall s \in S$$ 

The investment in capacity is again bounded. Similary to the bounds of the storage capacity it is limited by the existing capacity and the capacity potential.

$$ c_s^{capacity} \leq x_s^{capacity} \leq c_s^{capacity\_potential} $$



### Conversion component

A conversion unit will take from a bus and feed into another: 

$$x^{flow}_{c, to}(t) = c^{efficiencty}_{c} \cdot x^{flow}_{c, from}(t), \qquad \forall c  \in C, \forall t \in T$$ 

For the non-investment case the outflow, i.e. the flow **to** the bus where the conversion unit is connected with its output the following constraint applies:

$$x^{flow}_{c, to} \leq c^{capacity}_c$$

In the case of investment compare constraints from the volatile generator units.

In [13]:
for name, c in data["conversion"].iterrows():
    es.add(
        fc.Conversion(
            label=name,
            from_bus=buses[c.from_bus],
            to_bus=buses[c.to_bus],
            carrier=c.carrier,
            tech=c.tech,
            efficiency=c.efficiency,
            marginal_cost=technology.at[
                    (c["carrier"], c["tech"], tech_scenario), "vom"
                ],
            carrier_cost=carrier.at[(c.carrier, carrier_scenario), "cost"],
            expandable=c.expandable,
            capacity=c.capacity,
            capacity_potential=c.capacity_potential,
            capacity_cost=_capacity_cost(c, tech_scenario),
            output_parameters={
                "emission_factor": (
                    carrier.at[(c.carrier, carrier_scenario), "emission_factor"]
                    / c.efficiency)
            },
        )
    )


/home/admin/projects/oemof-barbados/barbados-env/lib/python3.7/site-packages/oemof/solph/network.py:323: SuspiciousUsageWarning: `Transformer` 'bagasse-st' constructed without `inputs`.
  warn(msg.format(self), debugging.SuspiciousUsageWarning)
/home/admin/projects/oemof-barbados/barbados-env/lib/python3.7/site-packages/oemof/solph/network.py:326: SuspiciousUsageWarning: `Transformer` 'bagasse-st' constructed without `outputs`.
  warn(msg.format(self), debugging.SuspiciousUsageWarning)
/home/admin/projects/oemof-barbados/barbados-env/lib/python3.7/site-packages/oemof/solph/network.py:323: SuspiciousUsageWarning: `Transformer` 'waste-ocgt' constructed without `inputs`.
  warn(msg.format(self), debugging.SuspiciousUsageWarning)
/home/admin/projects/oemof-barbados/barbados-env/lib/python3.7/site-packages/oemof/solph/network.py:326: SuspiciousUsageWarning: `Transformer` 'waste-ocgt' constructed without `outputs`.
  warn(msg.format(self), debugging.SuspiciousUsageWarning)


### Commodity components

For the commodity components, the aggregated flow for the complete time horizon is limited by the user defined amount:
$$ \sum_t x^{flow}_k(t) \leq c^{amount} \qquad \forall k \in K$$

In [14]:
for name, c in data["commodity"].iterrows():
    es.add(
        fc.Commodity(
            label=name,
            bus=buses[c.bus],
            carrier=c.carrier,
            tech=c.tech,
            amount=c.amount,
        )
    )


/home/admin/projects/oemof-barbados/barbados-env/lib/python3.7/site-packages/oemof/solph/network.py:263: SuspiciousUsageWarning: `Source` 'bagasse-commodity' constructed without `outputs`.
  warn(msg.format(self), debugging.SuspiciousUsageWarning)
/home/admin/projects/oemof-barbados/barbados-env/lib/python3.7/site-packages/oemof/solph/network.py:263: SuspiciousUsageWarning: `Source` 'waste-commodity' constructed without `outputs`.
  warn(msg.format(self), debugging.SuspiciousUsageWarning)


### Link components

The link components allows to model energy transfer from one bus to an other. For example transhipment modelling approach can be used for transmission in electricity systems. The equation for a line $n$ is considering a loss within the transfer process $from$ a bus $to$ the other bus.

$$x^{flow}_{from, n}(t) = c^{loss}_{n} \cdot x^{flow}_{n, to}(t), \qquad \forall n  \in N, \forall t \in T$$

In [15]:
for name, c in data["link"].iterrows():
    es.add(
        fc.Link(
            label=name,
            from_bus=buses[c.from_bus],
            to_bus=buses[c.to_bus],
            capacity=c.capacity,
            expandable=c.expandable,
             capacity_cost=annuity(
                technology.at[(c.carrier, c.tech, tech_scenario), "capex"],
                technology.at[
                    (c.carrier, c.tech, tech_scenario), "lifetime"
                ],
                technology.at[
                    (c.carrier, c.tech, tech_scenario), "wacc"
                ],
            )
            * 1000,  # $/kW -> $/MW
            loss=c.loss
        )
    )

### Objective Function 

The objective function is created from all instantiated objects. It will use all operating costs (i.e. `marginal_cost` argument) and if set all investment costs (i.e. `capacity_cost` argument) for all generators (dispatchable and volatile, conversion units and storages). In addition the energy related storage costs are added.  

$$ \text{min:} \sum_g \sum_t \overbrace{c^{marginal\_cost}_g \cdot x^{flow}_{g}(t)}^{\text{operating cost}} \\ 
\sum_g \sum_t \overbrace{c^{capacity\_cost}_g \cdot x^{capacity}_{g}(t)}^{\text{investment cost}} + \\
\sum_s \overbrace{c^{storage\_capacity\_cost}_s \cdot \overline{x}^{storage\_capacity}_{s}(t)}^{\text{storage energy cost}}$$

### Add Shortage/Excess Slack Components

If these are added, additional cost will occur inside the objective function. Shortage / Excess can be understood as a very expensive generator or load repectively. 

In [16]:
for name, e in data["excess"].iterrows():
    es.add(fc.Excess(label=name, bus=buses[e.bus]))


for name, s in data["shortage"].iterrows():
    es.add(
        fc.Shortage(
            label=name,
            carrier="electricity",
            tech="shortage",
            bus=buses[s.bus],
            marginal_cost=s.marginal_cost,
        )
    )


/home/admin/projects/oemof-barbados/barbados-env/lib/python3.7/site-packages/oemof/solph/network.py:250: SuspiciousUsageWarning: `Sink` 'el-excess' constructed without `inputs`.
  warn(msg.format(self), debugging.SuspiciousUsageWarning)


## Creating the Mathematical Model

In [17]:
# create model based on energy system and its components
m = Model(es)

### Add CO2 Constraint

To add a CO2-constraint we will use the `oemof.solph.constraints` module which allows to add such a constraint in a easy way. 

$$ \sum_t \sum_f  x^{flow}_f(t) \cdot c^{emission\_factor}_f \leq \overline{L_{CO_2}} $$

The constraint will sum all flows for the complete time horzion that have an attribute `emission_factor` and multiple the flow value with this factor. 

In [18]:
if co2_limit is not None:
    constraints.generic_integral_limit(
        m, keyword='emission_factor', 
        limit=co2_limit.loc["BB-electricity", "value"])
#m.write(io_options={"symbolic_solver_labels":True})

In [19]:
peak_demand = pd.concat([n.amount * n.profile for n in  es.nodes if isinstance(n, fc.Load)]).max()

def _excess_energy_limit(m):
    lhs = sum(m.flow[es.groups["BB-electricity"], es.groups["el-excess"], t] for t in m.TIMESTEPS)
    rhs = data["load"]["amount"].sum()
    return (lhs <= rhs * 0.1)
m.excess_energy_limit = po.Constraint(rule=_excess_energy_limit) 

def _excess_power_limit(m, t):
    lhs = m.flow[es.groups["BB-electricity"], es.groups["el-excess"], t]
    rhs = peak_demand
    return (lhs <= rhs)
m.excess_power_limit = po.Constraint(m.TIMESTEPS, rule=_excess_power_limit) 

In [20]:
dispatchable = [(i,o) for i,o in m.flows if isinstance(i, fc.Dispatchable) and not isinstance(i, fc.Shortage)] # all dispatchable are fossil
#conversion = [(i,o) for i,o in m.flows if isinstance(o, fc.Conversion)] 
demand = [(i,o) for i,o in m.flows if isinstance(o, fc.Load)]

In [21]:
m.receive_duals()

    with a new Component with type <class 'pyomo.core.base.suffix.Suffix'>.
    This is usually indicative of a modelling error. To avoid this warning,
    explicitly delete the attribute:
        del Model.dual
    with a new Component with type <class 'pyomo.core.base.suffix.Suffix'>.
    This is usually indicative of a modelling error. To avoid this warning,
    explicitly delete the attribute:
        del Model.rc


In [22]:
minimise_fossil_fuels = False
if minimise_fossil_fuels:
    # minimise conventional supply 
    m.del_component("objective")        
    m.objective =po.Objective(expr=sum(m.flow[i, o, t] for i,o in dispatchable for t in m.TIMESTEPS))

## Solving the Model and Writing Results

In [23]:
# check if cbc solver library is available
cbc = SolverFactory('cbc').available()

if cbc:
    #  solve the model using cbc solver
    m.solve("gurobi")

    # write results back to the model object
    m.results = m.results()
    
    # writing results with the standard oemof-tabular output formatt
    pp.write_results(m, m.results, scenario_path)
    
    _write_investment_cost(scenario_path)
    
    if co2_limit is not None:
        pd.Series(
            [(m.integral_limit_emission_factor_constraint() / 1e6), 
             m.dual[m.integral_limit_emission_factor_constraint], 
             m.objective()],
            index=["CO2 (Mio. t)", "Shadow Price in $/t", "Objective value"]
        ).to_csv(os.path.join(scenario_path, "costs.csv"))
    else: 
        pd.Series(
            [m.objective()],
            index=["Objective value"]
        ).to_csv(os.path.join(scenario_path, "costs.csv"))
    
    print(
        "Optimization done. Results are in {}.".format(
            results_path
        )
    )

Optimization done. Results are in /home/admin/oemof-results/barbados.


In [24]:
if minimise_fossil_fuels:
    max_share= 1 - m.objective() / pd.concat([n.amount * n.profile for n in  es.nodes if isinstance(n, fc.Load)]).sum()
    print(
            "Maximum RE share in scenario {0} is {1}.".format(
                scenario, max_share
            )
        )
    print("Fossil fuel supply: {} in GWh".format(m.objective()/1000))

### Print CO2-results

In [25]:
if co2_limit is not None:
    print("Emissions in Mio t: " + str(m.integral_limit_emission_factor_constraint() / 1e6))
    print("Shadow price of CO2: " + str(m.dual[m.integral_limit_emission_factor_constraint]))

Emissions in Mio t: 0.12787029430015284
Shadow price of CO2: 0.0


## Optional sensitivity calculations

To use this, adapt to your needs. 

In [ ]:
if True:
    #co2_base = m.integral_limit_emission_factor_constraint() # take as base for emissions
    sensitivities = [1] # 0.95 <=> 95% RE share
    for sensitivity in sensitivities:
        sensitivity_path = scenario_path + ("-" + str(int(sensitivity * 100)))
        
        if not os.path.exists(sensitivity_path):
            os.makedirs(sensitivity_path)
        
        m = Model(es)
        
        m.excess_energy_limit = po.Constraint(rule=_excess_energy_limit) 
        m.excess_power_limit = po.Constraint(m.TIMESTEPS, rule=_excess_power_limit) 
    
        
        def _re_share(m):
            lhs = sum(m.flow[i, o, t] for i,o in dispatchable for t in m.TIMESTEPS)
            rhs = sum(m.flow[i, o, t] for i,o in demand for t in m.TIMESTEPS)
            return (lhs <= (1 - sensitivity) * rhs)
        m.renewable_share = po.Constraint(rule=_re_share)  
        
        #constraints.generic_integral_limit(
        #    m, keyword='emission_factor', 
        #    limit=sensitivity * co2_base)
        
        m.receive_duals()

        m.solve("gurobi")

        m.results = m.results()

        pp.write_results(m, m.results, sensitivity_path)
        
        _write_investment_cost(sensitivity_path)
        
        pd.Series(
            [m.objective()],
            index=["Objective value"]
        ).to_csv(os.path.join(sensitivity_path, "costs.csv"))
            
        #pd.Series(
        #    [(m.integral_limit_emission_factor_constraint() / 1e6), 
        #     m.dual[m.integral_limit_emission_factor_constraint], 
        #     m.objective()],
        #    index=["CO2 (Mio. t)", "Shadow Price in $/t", "Objective value"]
        #).to_csv(os.path.join(sensitivity_path, "costs.csv))

on block (model).Model with a new Component
with type <class 'pyomo.core.base.suffix.Suffix'>.
This is usually indicative of a modelling error.
To avoid this warning, explicitly delete the attribute:
    del Model.dual
on block (model).Model with a new Component
with type <class 'pyomo.core.base.suffix.Suffix'>.
This is usually indicative of a modelling error.
To avoid this warning, explicitly delete the attribute:
    del Model.rc


    with a new Component with type <class 'pyomo.core.base.suffix.Suffix'>.
    This is usually indicative of a modelling error. To avoid this warning,
    explicitly delete the attribute:
        del Model.dual
    with a new Component with type <class 'pyomo.core.base.suffix.Suffix'>.
    This is usually indicative of a modelling error. To avoid this warning,
    explicitly delete the attribute:
        del Model.rc


## Visualization

### Installed capacities

In [ ]:
#scenario_path = sensitivity_path

In [ ]:
capacities = pd.read_csv(os.path.join(scenario_path, "capacities.csv"), index_col=[0,1,2,3,4])
ax = capacities.xs(("BB-electricity", "invest"), level=[1,2]).droplevel([1,2]).plot(kind="bar", rot=90)
ax.set_xlabel("Technology")
ax.set_ylabel("Installed capacity in MW")

In [ ]:
filling = pd.read_csv(os.path.join(scenario_path, "filling_levels.csv"), index_col=[0], parse_dates=True)
ax = filling.plot()
ax.set_ylabel("Energy in MWh")
ax.set_xlabel("Hour of the year")

In [ ]:
balance = pd.read_csv(os.path.join(scenario_path, "BB-electricity.csv"), index_col=[0], parse_dates=True)
ax = (balance.sum() / 1e3).plot(kind="barh",rot=0)
ax.set_xlabel("Enegy in Gwh")
ax.set_ylabel("Technology")

In [ ]:
sums = balance.sum() / 1e6

re_share = (
    sums[[i for i in all_components.index if "onshore" in i]].sum() + 
    sums[[i for i in all_components.index if "pv" in i]].sum() + 
    sums["bagasse-st"] - 
    sums["el-excess"]
    ) / sums[[i for i in all_components.index if "load" in i]].sum()
print("Renewable energy share is {} %".format(100 * round(re_share,4)))

### LCOE

In [ ]:
lcoe = (
    m.objective() / (
        (sums[[i for i in all_components.index if "load" in i]].sum()
         - sums["el-excess"]) * 1e6))
print("LCOE are {} BBD/kWh".format(round(lcoe / 1000, 3)))

### Capacity factors

In [ ]:
build = capacities.xs(("BB-electricity", "invest"), level=[1,2]).droplevel([1,2])
(balance.sum()[build.index] / (build * 8760)["value"]).plot(kind="bar")